In [34]:
from bs4 import BeautifulSoup
import requests
page_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response=requests.get(page_link,timeout=5)
soup = BeautifulSoup(page_response.content,"html.parser")
#print(soup.prettify())

In [35]:
a=soup.find_all('td')
#print(len(a))
postcode=[]
borough=[]
neighbor=[]
for i in range(0,len(a)-2,3):
    postcode.append(a[i].get_text().strip())
    i=i+1
    borough.append(a[i].get_text().strip())
    i=i+1
    neighbor.append(a[i].get_text().strip()+',')

import pandas as pd
location=pd.DataFrame({'postcode' : postcode,
              'borough' : borough,
              'neighbor' : neighbor}, 
              columns=['postcode','borough', 'neighbor'])
#location

In [36]:
mask = (location['borough'] != 'Not assigned') & (location['postcode'].str.len() == 3)
location2=location.loc[mask].sort_values('postcode').reset_index().drop('index',axis=1)

## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [37]:
location2=location2.groupby(["postcode", "borough"], as_index=False).sum()
location2['neighbor'] = location2['neighbor'].astype(str).str[:-1]
#location2

## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [38]:
mask=(location2['neighbor'] == 'Not assigned')
location2.loc[mask,'neighbor']=location2.loc[mask,'borough']
location2

,postcode,borough,neighbor
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
8,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [39]:
location2.shape

(103, 3)

In [40]:
df=pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv").rename(columns={"Postal Code":"postcode"})
df.head()

,postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
df2=pd.merge(location2, df, on='postcode')
df2.head()

,postcode,borough,neighbor,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
